## step 1: preprocess the data, including adding the header,remove the null data 

In [119]:
import pandas as pd
## load the train and test data into dataframe
## import the train data without header
train = pd.read_excel('train.xlsx', header=None,encode='utf-8')
## import the test data 
test = pd.read_excel('test.xlsx',encode='utf-8')

In [120]:
import dataprocesor
reload(dataprocesor)
from dataprocesor import DataProcessor

In [121]:
## initialize the dataprocess class
dp = DataProcessor(train,test)

In [122]:
print 'the raw information for the train data'

the raw information for the train data


In [123]:
train.head()

,0,1,2,3,4
0,--- X 10 --- 七色 多層次搭配 圓下擺 LAYERED 素面 無袖背心 打底,Male Fashion,無袖,Impression,31/7/17
1,︱IBIT︱Gymshark 熱銷款 運動T恤 健身T恤 圓領短T 運動短T 健身鯊魚,Male Fashion,gymshark,Impression,31/7/17
2,︱IBIT︱Gymshark 超高彈性 短褲 運動短褲 跑步短褲 深蹲褲 訓練短褲,Male Fashion,gymshark,Impression,31/7/17
3,::另類情侶兄弟姊妹殼::電力滿格/不足黑白趣味浮雕手機軟殼i5/i5s/i5se/i6/i...,Mobile & Gadgets,軟殼,Click,31/7/17
4,：新舊手機商場：Iphone6 16金 （需要看細圖密我）,Mobile & Gadgets,iphone6 系列,Impression,30/7/17


In [124]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10317 entries, 0 to 10316
Data columns (total 5 columns):
0    10304 non-null object
1    10317 non-null object
2    10317 non-null object
3    10317 non-null object
4    10317 non-null object
dtypes: object(5)
memory usage: 403.1+ KB


In [125]:
### data cleaning, include removing null data, adding the column title
train = dp.preprocess()

In [126]:
print 'after process, the train data information'

after process, the train data information


In [127]:
train.head()

,Product Name,Category,Query,Event,Date
0,--- X 10 --- 七色 多層次搭配 圓下擺 LAYERED 素面 無袖背心 打底,Male Fashion,無袖,Impression,31/7/17
1,︱IBIT︱Gymshark 熱銷款 運動T恤 健身T恤 圓領短T 運動短T 健身鯊魚,Male Fashion,gymshark,Impression,31/7/17
2,︱IBIT︱Gymshark 超高彈性 短褲 運動短褲 跑步短褲 深蹲褲 訓練短褲,Male Fashion,gymshark,Impression,31/7/17
3,::另類情侶兄弟姊妹殼::電力滿格/不足黑白趣味浮雕手機軟殼i5/i5s/i5se/i6/i...,Mobile & Gadgets,軟殼,Click,31/7/17
4,：新舊手機商場：Iphone6 16金 （需要看細圖密我）,Mobile & Gadgets,iphone6 系列,Impression,30/7/17


In [128]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10303 entries, 0 to 10303
Data columns (total 5 columns):
Product Name    10303 non-null object
Category        10303 non-null object
Query           10303 non-null object
Event           10303 non-null object
Date            10303 non-null object
dtypes: object(5)
memory usage: 483.0+ KB


## Step 2: Continue to process the train data
#### Step 2.1 segment the Product name using jieba

In [129]:
train['product_fenci'] = dp.jieba_fenci(train)
train['Product Seg'] = map(lambda x: dp.remove_space(x.split('|')), train['product_fenci'])
train.drop('product_fenci',1, inplace=True)

In [130]:
train.head()

,Product Name,Category,Query,Event,Date,Product Seg
0,--- X 10 --- 七色 多層次搭配 圓下擺 LAYERED 素面 無袖背心 打底,Male Fashion,無袖,Impression,31/7/17,"[X, 10, 七色, 多層次, 搭配, 圓下, 擺, LAYERED, 素面, 無袖, 背..."
1,︱IBIT︱Gymshark 熱銷款 運動T恤 健身T恤 圓領短T 運動短T 健身鯊魚,Male Fashion,gymshark,Impression,31/7/17,"[IBIT, Gymshark, 熱銷款, 運動, T恤, 健身, T恤, 圓領, 短, T..."
2,︱IBIT︱Gymshark 超高彈性 短褲 運動短褲 跑步短褲 深蹲褲 訓練短褲,Male Fashion,gymshark,Impression,31/7/17,"[IBIT, Gymshark, 超高, 彈性, 短褲, 運動, 短褲, 跑步, 短褲, 深..."
3,::另類情侶兄弟姊妹殼::電力滿格/不足黑白趣味浮雕手機軟殼i5/i5s/i5se/i6/i...,Mobile & Gadgets,軟殼,Click,31/7/17,"[另類, 情侶, 兄弟, 姊妹, 殼, 電力, 滿格, 不足, 黑白, 趣味, 浮雕, 手機..."
4,：新舊手機商場：Iphone6 16金 （需要看細圖密我）,Mobile & Gadgets,iphone6 系列,Impression,30/7/17,"[新舊, 手機, 商場, Iphone6, 16, 金, 需要, 看細, 圖密, 我]"


## Step 2 : Continue to process the train data
#### Step 2.2.   evaluation the query quality

In [131]:
### event count for the query 
query_term_all  = list(train['Query'].values)
## to total count number when the particular query appeared as 'impression' 
train['Impression No'] = dp.event_count(u'Impression', query_term_all)
## to total count number when the particular query appeared as 'click' 
train['Click No'] = dp.event_count(u'Click',query_term_all)
## the ctr represent the quality of the query
train['CTR']= train['Click No']/(train['Click No']+ train['Impression No']) 

In [132]:
train.head()

,Product Name,Category,Query,Event,Date,Product Seg,Impression No,Click No,CTR
0,--- X 10 --- 七色 多層次搭配 圓下擺 LAYERED 素面 無袖背心 打底,Male Fashion,無袖,Impression,31/7/17,"[X, 10, 七色, 多層次, 搭配, 圓下, 擺, LAYERED, 素面, 無袖, 背...",16,2,0.111111
1,︱IBIT︱Gymshark 熱銷款 運動T恤 健身T恤 圓領短T 運動短T 健身鯊魚,Male Fashion,gymshark,Impression,31/7/17,"[IBIT, Gymshark, 熱銷款, 運動, T恤, 健身, T恤, 圓領, 短, T...",5,0,0.000000
2,︱IBIT︱Gymshark 超高彈性 短褲 運動短褲 跑步短褲 深蹲褲 訓練短褲,Male Fashion,gymshark,Impression,31/7/17,"[IBIT, Gymshark, 超高, 彈性, 短褲, 運動, 短褲, 跑步, 短褲, 深...",5,0,0.000000
3,::另類情侶兄弟姊妹殼::電力滿格/不足黑白趣味浮雕手機軟殼i5/i5s/i5se/i6/i...,Mobile & Gadgets,軟殼,Click,31/7/17,"[另類, 情侶, 兄弟, 姊妹, 殼, 電力, 滿格, 不足, 黑白, 趣味, 浮雕, 手機...",1,1,0.500000
4,：新舊手機商場：Iphone6 16金 （需要看細圖密我）,Mobile & Gadgets,iphone6 系列,Impression,30/7/17,"[新舊, 手機, 商場, Iphone6, 16, 金, 需要, 看細, 圖密, 我]",38,12,0.240000


## Step3 :  Create a similarity model 
#### first need to create a dictionary and corpus, and then covert the corpus to vector in tfidf space, finally build a similarity index model

In [133]:
index = dp.get_index()

generating the dictionary
Dictionary(22053 unique tokens: [u'\u9ede\u96ea\u7d21\u7d17', u'\u7b2c\u4e8c', u'\u5c0f\u817f', u'\u5e73\u5b89', u'\u5bec\u7248\u6d0b\u88dd']...)
tfidf model info: TfidfModel(num_docs=10303, num_nnz=160034)
the maximum len of the vector 50
the average len of the vector 15.5327574493


## Step 4  Raise a query of product, get the similar product list
#### process the test data

In [134]:
test ['product_fenci'] = dp.jieba_fenci(test)
test['Product Seg'] = map(lambda x: dp.remove_space(x.split('|')), test['product_fenci'])
test.drop('product_fenci',1, inplace=True)
test['Product Seg'] = map(lambda x: dp.lower_case(x), test['Product Seg'])

In [135]:
test.head()

,Product Name,Category,Product Seg
0,寬鬆顯瘦大碼運動套裝T恤女夏季胖mm短袖短褲時尚休閒服兩件套,Female Clothes,"[寬, 鬆, 顯, 瘦, 大, 碼, 運, 動, 套, 裝, t恤, 女, 夏季, 胖, m..."
1,♬【現貨實拍】夏季新款 2017韓版熱銷淑女夏裝間約氣質條紋背心吊帶連體褲顯瘦闊腿褲,Female Clothes,"[現貨實, 拍, 夏季, 新款, 2017, 韓版, 熱銷, 淑女, 夏裝間, 約, 氣質,..."
2,新款時尚大碼女士服裝韓版印花短袖 t恤女夏寬鬆顯瘦,Female Clothes,"[新款, 時尚, 大碼, 女士, 服裝, 韓版, 印花, 短袖, t, 恤, 女夏, 寬, ..."
3,a la sha 粉紅色阿財長版上衣,Female Clothes,"[a, la, sha, 粉紅色, 阿財長, 版, 上衣]"
4,女人的店~上班短裙.包臀裙.西裝裙(垂性很好.不易皺.不起球.不沾毛) 350元,Female Clothes,"[女人, 的, 店, 上班, 短裙, 包, 臀, 裙, 西裝, 裙, 垂性, 很, 好, 不..."


In [136]:
### recommend keywords for the product from test data 
test['keyword recommend']= ''
### configurable parameters
num_similar_product_list = 40 #### number of similar product you want to get
num_keyword = 2 ### the number of keywords to show 
weight = [0.1, 0.1, 0.2,0.1, 0.4, 0.1] ## [query_count,ctr,similrity,click,relevance,event]
column_list = ['Event','Query','Impression No', 'Click No','CTR'] 

for row in test.iterrows():
    row_location = row[0]
    ### retrieve the produt segment
    product_texts_cuts = test['Product Seg'][row_location]
    ### get the similar product list
    similar_product = dp.get_similar_product_list(product_texts_cuts,num_similar_product_list, column_list)
    ### add more evaluators for the query
    similar_product_evaluate = dp.get_evaluate_product_list(weight,similar_product,product_texts_cuts,row_location)
    ### recommend keywords according the total score of the keywords
    test_submit = dp.get_recommend_keyword(similar_product_evaluate,num_keyword,row_location)

In [137]:
test_submit.head()

,Product Name,Category,Product Seg,keyword recommend
0,寬鬆顯瘦大碼運動套裝T恤女夏季胖mm短袖短褲時尚休閒服兩件套,Female Clothes,"[寬, 鬆, 顯, 瘦, 大, 碼, 運, 動, 套, 裝, t恤, 女, 夏季, 胖, m...","寬鬆短袖,運動套裝"
1,♬【現貨實拍】夏季新款 2017韓版熱銷淑女夏裝間約氣質條紋背心吊帶連體褲顯瘦闊腿褲,Female Clothes,"[現貨實, 拍, 夏季, 新款, 2017, 韓版, 熱銷, 淑女, 夏裝間, 約, 氣質,...","吊帶褲,吊帶褲 吊帶長褲"
2,新款時尚大碼女士服裝韓版印花短袖 t恤女夏寬鬆顯瘦,Female Clothes,"[新款, 時尚, 大碼, 女士, 服裝, 韓版, 印花, 短袖, t, 恤, 女夏, 寬, ...","短袖t恤,寬鬆短袖"
3,a la sha 粉紅色阿財長版上衣,Female Clothes,"[a, la, sha, 粉紅色, 阿財長, 版, 上衣]","a la sha,a la sha 褲裙"
4,女人的店~上班短裙.包臀裙.西裝裙(垂性很好.不易皺.不起球.不沾毛) 350元,Female Clothes,"[女人, 的, 店, 上班, 短裙, 包, 臀, 裙, 西裝, 裙, 垂性, 很, 好, 不...","包臀裙,裙"


In [140]:
### save the result to csv file
column = ['Product Name','Category','keyword recommend']
test_submit[column].to_csv('test_submit.csv', index=False, encoding='utf-8')


#### try lsi model

In [24]:
'''''''''
#### take long time to run the lsi model
### try lsi model
### recommend the keyword for the product from test data 
test['keyword recommend']= ''

num_similar_product_list = 20 #### number of similar product you want to get
number_keyword = 2 ### the number of keywords to show 

column_list = ['Query','Impression No', 'Click No','CTR'] 

for row in test[0:10].iterrows():
    row_location = row[0]
    ### retrieve the produt segment
    product_texts_cuts = test['Product Seg'][row_location]
    ### get the similar product list
    similar_product = dp.get_similarity_product_list_lsi(product_texts_cuts,num_similar_product_list, column_list)
''''''

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-24-23421fbc6557>, line 18)